In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [7]:
import datetime
import math
import time
import pandas as pd
import requests
import json
from pandas.io.formats.style import Styler

In [8]:
options = webdriver.ChromeOptions ()
options.add_argument ('--headless')
options.add_experimental_option ('excludeSwitches', ['disable-popup-blocking'])
options.add_argument ('--window-size=1920x1080')
services = webdriver.chrome.service.Service() # (CHROME_DRIVER_ADDRESS)

In [65]:
P41_ASIN = 'B09QX6SL2Y'
Fire_HD_10_Plus_ASIN = 'B08F6FYN6B'

In [66]:
DAYS = 3 # Scrape from DAYS days ago

def parse_review(review1):
    name1 = review1.find_element (By.XPATH, ".//span[@class='a-profile-name']").text
    star1 = review1.find_element (By.XPATH, ".//i[@data-hook='cmps-review-star-rating' or @data-hook='review-star-rating']").get_attribute('class')
    star1 = str( star1.split (' ')[2][-1])[0]
    title1 = review1.find_element (By.XPATH, ".//*[@data-hook='review-title']").text
    date1 = review1.find_element (By.XPATH, ".//span[@data-hook='review-date']").text
    url1 = review1.find_elements (By.XPATH, "./div/div/div/a")[1].get_attribute('href')
    link1 = '<a href = "' + url1 + '">Text</a>'
    if date1[-3:] == '검토됨':
        date1 = date1[5:-5]
        date1 = datetime.datetime.strptime(date1,'%Y년 %m월 %d일')
    else:
        date1 = datetime.datetime.strptime(date1.split('on ')[1],'%B %d, %Y')
    if (datetime.datetime.today() - date1).days >= DAYS:
        return 'Stop'
    date1 = date1.strftime("%m/%d")
    format1 = review1.find_elements(By.XPATH , ".//a[@data-hook='format-strip']")
    purchased1 = review1.find_elements(By.XPATH , ".//span[@data-hook='avp-badge']")
    if len(purchased1) == 1:
        purchased2 = purchased1[0].text
    else:
        purchased2 = 'No'
    text1 = review1.find_element(By.XPATH, ".//span[@data-hook='review-body']").text
    if 'Translate review to English' in review1.text :
        text1 = text1 # Use any translation function you can use
        title1 = title1 # Use any translation function you can use
    images0 = review1.find_elements(By.XPATH, ".//div[@class='review-image-tile-section']//img")
    if len(images0) >= 1:
        imgurl = [x.get_attribute('src')[:-9] + 'jpg' for x in images0]
        imglink = ['<a href = "' + x + '">Img</a>' for x in imgurl]
    else:
        imglink = ['']
        textimglink = ' '.join([link1] + imglink)
    helpful1 = review1.find_elements(By.XPATH , ".//span[@data-hook='helpful-vote-statement']")
    if len(helpful1) == 1:
        helpful2 = helpful1[0].text
    else:
        helpful2 = str(0)
    if len(review1.find_elements(By.XPATH, ".//span[@class='a-size-base']")) >= 1:
        nocomment1 = review1.find_element(By.XPATH, ".//span[@class='a-size-base']").text
    else:
        nocomment1 = 'None'
    
    review_dict = {'Name': name1,'Date':date1,'Link': textimglink, 'Rating': star1,'Title': title1,'Capacity': format1,'Purchased': purchased2,
                'Text': text1,'Num_of_Images': len(images0), 'Helpful': helpful2,'Num_of_Comments': nocomment1}
    return review_dict

def scrape_reviews(asin, show = False, allformats = True):
    p_num = 0
    reviews_df = pd.DataFrame()
    while True:
        p_num += 1
        url1 = 'https://www.amazon.com/product-reviews/' + asin + '/ref=cm_cr_arp_d_viewopt_srt?language=en_US&formatType='
        url3 = '&pageNumber=' + str(p_num) + '&sortBy=recent'
        if allformats == True:
            url4 = url1 + 'all_formats' + url3
        else:
            url4 = url1 + 'current_format' + url3
        driver = webdriver.Chrome (service=services, options=options)
        driver.get(url4)
        reviews_list = WebDriverWait(driver, 10).until(lambda driver: driver.find_elements(By.XPATH , "//div[@data-hook='review']"))
        print(f"Num. of reviews: {len(reviews_list)}")
        if len(reviews_list) == 0:
            print("No more reviews. Finished")
            break
        if p_num % 10 == 0:
            print(f'Page {p_num}', end= ' ')
        for review1 in reviews_list:
            review_dict = parse_review(review1)
            if review_dict == 'Stop':
                driver.close()
                driver.quit()
                return reviews_df
            review_df = pd.DataFrame([review_dict.values()], columns = review_dict.keys ())
            reviews_df = pd.concat([reviews_df, review_df])
    driver.close()
    driver.quit()
    return reviews_df

In [67]:
result1 = scrape_reviews(Fire_HD_10_Plus_ASIN)
result1

Num. of reviews: 10


,Name,Date,Link,Rating,Title,Capacity,Purchased,Text,Num_of_Images,Helpful,Num_of_Comments
0,Imfh450,11/06,"<a href = ""https://www.amazon.com/gp/customer-...",3,It’s ok,[<selenium.webdriver.remote.webelement.WebElem...,Verified Purchase,"It’s alright, gets the job done for kids",0,0,None
0,Jack Page,11/04,"<a href = ""https://www.amazon.com/gp/customer-...",5,The product is as described,[<selenium.webdriver.remote.webelement.WebElem...,Verified Purchase,The product is an excellent alternative to the...,0,0,None
0,Joyce Urquhart,11/04,"<a href = ""https://www.amazon.com/gp/customer-...",5,So glad I made this purchase!,[<selenium.webdriver.remote.webelement.WebElem...,Verified Purchase,After looking at so many different tables and ...,0,0,None
0,Quentin Lesh,11/04,"<a href = ""https://www.amazon.com/gp/customer-...",5,Best one yet,[<selenium.webdriver.remote.webelement.WebElem...,Verified Purchase,Love how this has wireless charging capabiliti...,0,One person found this helpful,None
